## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [14]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=0.99,
                 criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split  # ограничение минимального числа объектов в листе
        self.max_depth = max_depth  # параметр max_depth ограничивает глубину дерева
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    # Набор функция для вычисления меры "нечистоты" (Impurity)
    # или меры неоднородности при разделении в узле дерева на левый и правый узлы
    # В функциях используются следующие входные параметры:
    # l_c == l_class_count - количество образцов первого и второго класса в левой части разделённого массива
    # l_s == l_sizes - размер левой части
    # r_c == r_class_count - количество образцов первого и второго класса в правой части разделённого массива
    # r_s == r_sizes - размер правой части
    # передаются массивы таких значений для всех возможных разбиений по местам изменения целевого признака
    # возвращают значения меры неоднородности для различных возможных разбиений
    
    
    # мера неоднородности Джини
    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        # Общий размер левой и правой части
        size_all = l_s[0, 0] + r_s[0, 0]
        # Доля образцов, котороые принадлежат классу 1 в левом узле
        p_1_left = l_c[:, 0] / l_s[:, 0]
        # Доля образцов, котороые принадлежат классу 2 в левом узле
        p_2_left = l_c[:, 1] / l_s[:, 0]
        # Мера Джини для левого узла
        gini_left = 1.0 - np.square(p_1_left) - np.square(p_2_left)
        # Аналогично для правого узла
        p_1_right = r_c[:, 0] / r_s[:, 0]
        p_2_right = r_c[:, 1] / r_s[:, 0]
        gini_right = 1.0 - np.square(p_1_right) - np.square(p_2_right)
       
        gini_sum = (l_s[:, 0] / size_all) * gini_left + (r_s[:, 0] / size_all) * gini_right      
        return gini_sum
    
    
    # Информационная энтропия
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        threshold = 1e-10
        # Общий размер левой и правой части
        size_all = l_s[0, 0] + r_s[0, 0]
        # Доля образцов, котороые принадлежат классу 1 в левом узле
        p_1_left = l_c[:, 0] / l_s[:, 0]
        # Доля образцов, котороые принадлежат классу 2 в левом узле
        p_2_left = l_c[:, 1] / l_s[:, 0]
        # Энтропия для левого узла
        entropy_left = -1.0 * (p_1_left * np.log2(np.clip(p_1_left, threshold, 1.0 - threshold)) +
                               p_2_left * np.log2(np.clip(p_2_left, threshold, 1.0 - threshold)))
        # Аналогично для правого узла
        p_1_right = r_c[:, 0] / r_s[:, 0]
        p_2_right = r_c[:, 1] / r_s[:, 0]
        entropy_right = -1.0 * (p_1_right * np.log2(np.clip(p_1_right, threshold, 1.0 - threshold)) +
                                p_2_right * np.log2(np.clip(p_2_right, threshold, 1.0 - threshold)))
       
        entropy_sum = (l_s[:, 0] / size_all) * entropy_left + (r_s[:, 0] / size_all) * entropy_right      
        return entropy_sum

    
    # Ошибка классификации (misclassification error)
    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        # Общий размер левой и правой части
        size_all = l_s[0, 0] + r_s[0, 0]
        # Неоднородность левой части после разделения
        # (выбираем преобладающий класс в левой части и делим на размер левой части == максимальная
        # вероятность класса в левой части после разделения)
        ME_left = 1.0 - np.max(l_c, axis=1) / l_s[:,0]
        # Неоднородность правой части после разделения
        ME_right = 1.0 - np.max(r_c, axis=1) / r_s[:,0]
        
        # Общая неоднородность левой и правой частей
        ME_sum = (l_s[:, 0] / size_all) * ME_left + (r_s[:, 0] / size_all) * ME_right   
        return ME_sum

    
    # Дла разделения в узлах могут быть выбираны не все признаки, а только несколько из них
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return # Ваш код
      
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return # Ваш код

    
    def __get_feature_ids_N(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids
 

    # Выполняет сортировку признака по возрастанию. На вход передаётся один признак x
    def __sort_samples(self, x, y):  
        sorted_idx = x.argsort() 
        return x[sorted_idx], y[sorted_idx]

    
    # метод делит выборку по пороговому значению
    # выбираются те строки, где значение признака feature_id больше порогового значения (left_mask),
    # и те строки, где значение признака feature_id меньше порогового значения (rigth_mask)
    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    
    # Метод находит пороги (т.е. значения признака, при которых целевой признак y
    # меняет своё значение) для признака x и выбирает из них наилучшей с точки зрения
    # минимизации меры неопределённости
    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # 1. Сортирует x и y в порядке возрастания x
        # 2. Вычисляется общее количество классов в y (в нашем случае 2 класса - 0 и 1)
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        # 1. Выбираем из массива sorted_y значения в середине, кроме первых и последних min_samples_split
        # строк (чтобы соблюсти ограничение на минимальное кол-во точек в листе дерева)
        # 2. В np.where(...) мы находим индексы в массиве y, в которых значение y изменяется
        # (сравнивается массив y и он же, но смещённый на одну позицию. Т.е. каждый элемент y сравнивается
        # со следующим элементом y). Далее индекс приводится к позиции в исходной матрице.
        # В итоге r_border_ids содержит индекс элемента в массиве sorted_y, в котором значение y изменилось
        # по сравнению со значением в предыдущем элементе
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        # Если для данного признака x целевой признак ни разу не изменяется, то такой признак 
        # не имеет смысла рассматривать для построения дерева
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        # 1. eq_el_count - вычисляется, как долго в массиве y сохраняется постоянное значение до очередного изменения
        # 2,3. one_hot_code - строит массив из двух столбцов с чередующимися 0 и 1, смещёнными на одну позицию
        # в первом и втором столбце (своеобразная "змейка")
        # 4. Первый столбец one_hot_code умножается на столбец eq_el_count
        # 5. К первой строке ещё добавляется поправка на количество разных классво в первых 
        # min_samples_split строках sorted_y
        # В итоге в class_increments содержится таблица, в которой показано, сколько раз сохраняется каждое
        # значение в sorted_y перед следующим изменением
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода?
        # Вычисляются следующие показатели:
        # l_class_count - количество элементво 1-го и 2-го классов в левой части разделённого массива
        # r_class_count - количество элементов 1-го и 2-го классов в правой части разделённого массива
        # l_sizes = размер левой половины разделённого массива
        # r_sizes = размер правой половины разделённого массива
        # и, т.к. разделение возможно во всех местах изменения целевого признака y, то эти показатели
        # вычисляются для всех мест разделения
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # 1. Вычисляется мера неоднородности при разделении класса как сумма неоднородностей
        # в левой и правой половинах
        # 2. Индекс разделения, дающего минимальную неоднородность (лучшее разделение)
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
    
        # Что делает этот блок кода?
        # 1. Размер левой половины разделённого массива
        # 2. Возвращает значение меры неоднородности, а также среднее значение признака x
        # слева и справа от границы разделения (пороговое значение)
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    
    # Метод, который рекурсивно строит дерево решений
    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # В начале проверяем, не достигнуто ли условие окончания построения дерева
        # (окончание рекурсивных вызовов)
        
        # 1. Лист ли это?
        is_it_leaf = False
        leaf_reason = 0  # причина, по которой мы определяем, что это лист (для отладки)
        
        # достигнута максимальная глубина дерева
        if (self.max_depth is not None) and (depth >= self.max_depth):
            is_it_leaf = True
            leaf_reason = 1
            
        # достигнуто минимальное количество образцов для узла дерева    
        if x.shape[0] <= self.min_samples_split:
            is_it_leaf = True
            leaf_reason = 2
            
        # если в узле доля одного класса больше порогового значения, 
        # то тогда не будем его дальше делить
        main_class_shape = np.max(np.bincount(y))*1.0 / y.shape[0]
        if main_class_shape >= self.sufficient_share:
            is_it_leaf = True
            leaf_reason = 3
        
        if is_it_leaf:  # создаём лист
            self.tree[node_id] = self.__create_leaf_node(y, leaf_reason)
            return
        else:  # иначе создаём узел и разделяем его
            # выбираем признаки для разделения (берём все столбцы из x)
            features_ids = self.get_feature_ids(x.shape[1])  

            # находим из всех признаков такой, разделение по которому обеспечивает
            # наименьшую неопределённость (наибольший прирост информации)
            impurity_list = []
            threshold_list = []
            feature_id_list = []
            
            for feature_id in features_ids:
                impurity, threshold = self.__find_threshold(x[:, feature_id], y)
                # Если threshols == None, то для данного признака x целевой признак ни разу не изменяется
                if threshold is not None:
                    impurity_list.append(impurity)
                    threshold_list.append(threshold)
                    feature_id_list.append(feature_id)

            if len(impurity_list) == 0:  # тогда это лист
                leaf_reason = 4
                self.tree[node_id] = self.__create_leaf_node(y, leaf_reason)
                return
            
            # индекс "наилучшего" признака, соответствующего минимальной impurity
            min_impurity = min(impurity_list)
            min_impurity_index = impurity_list.index(min_impurity)
            # id наилучшего признака
            min_impurity_feature_id = feature_id_list[min_impurity_index]      
            # порог для разделения
            feature_threshold = threshold_list[min_impurity_index]
            
            # разделяем данные в узле
            x_left, x_right, y_left, y_right = self.__div_samples(x, y, min_impurity_feature_id, feature_threshold)
            
            # если после разделения в одной из половин нет данных, то вторая половина - это лист
            # (это одно из условий окончания рекурсии)
            if y_left.shape[0] == 0 and y_right.shape[0] == 0:
                return
            if y_left.shape[0] == 0:
                leaf_reason = 5
                self.tree[node_id] = self.__create_leaf_node(y_right, leaf_reason)
                return
            if y_right.shape[0] == 0:
                leaf_reason = 6
                self.tree[node_id] = self.__create_leaf_node(y_left, leaf_reason)
                return
            # иначе создаём промежуточный узел и выполняем рекурсивный вызов для левого и правого поддеревьев
            if y_left.shape[0] > 0 and y_right.shape[0] > 0:
                self.tree[node_id] = self.__create_non_leaf_node(min_impurity_feature_id, feature_threshold, min_impurity, impurity_list)
                # self.__check_main_class_left_and_right(x, y, y_left, y_right, min_impurity_feature_id, feature_threshold)
                self.__fit_node(x_left, y_left, 2 * node_id + 1, depth + 1)
                self.__fit_node(x_right, y_right, 2 * node_id + 2, depth + 1)
   

    def __create_non_leaf_node(self, feature_id, threshold, min_impurity, impurity_list):
        node = dict()
        node[0] = self.__class__.NON_LEAF_TYPE
        # feature_id 
        node[1] = feature_id
        # threshold
        node[2] = threshold
        node[3] = min_impurity
        node[4] = impurity_list
        return node
        
    
    def __create_leaf_node(self, y, reason):
        node = dict()
        node[0] = self.__class__.LEAF_TYPE
        # назначаем листу класс - наиболее часто встречающуюся в y величину
        node[1] = self.__get_main_class(y)
        # назначаем в листе долю основного класса
        node[2] = np.max(np.bincount(y))*1.0 / y.shape[0]
        # причина создания листа (для отладки)
        node[3] = reason
        # количество образцов в листе (нужно для слияния)
        node[4] = y.shape[0]
        return node
    
    
    def __check_main_class_left_and_right(self, x_before_split, y_before_split, y_left, y_right, min_impurity_feature_id, feature_threshold):
        main_class_left = self.__get_main_class(y_left)
        main_class_right = self.__get_main_class(y_right)
        if main_class_left == main_class_right:
            print("Совпадают main классы после разделения")
            print("Main class")
            print(main_class_left)
            print("y_left")
            print("size = {}".format(y_left.shape[0]))
            print(y_left)
            print("y_right")
            print("size = {}".format(y_right.shape[0]))
            print(y_right)
            print("min_impurity_feature_id")
            print(min_impurity_feature_id)
            print("feature_threshold")
            print(feature_threshold)
            print("Колонка признака до разделения")
            print(x_before_split[:, min_impurity_feature_id])
            print("Целевой признак до разделения")
            print(y_before_split)
            print("")
    
    
    def __get_main_class(self, y):
        return np.argmax(np.bincount(y))
            
        
    # после построения дерева сливаем листы потомки, если они дети одного
    # родителя и имеют одинаковый класс
    def __merge_leafs(self):
        self.__walk_tree_and_merge_leafs(0)
    
    
    def __walk_tree_and_merge_leafs(self, node_id):
        if node_id in self.tree:       
            node = self.tree[node_id]
            if node[0] == self.__class__.NON_LEAF_TYPE:    
                left_node_id = 2 * node_id + 1
                right_node_id = 2 * node_id + 2
                if left_node_id in self.tree and right_node_id in self.tree:
                    left_node = self.tree[left_node_id]
                    right_node = self.tree[right_node_id]
                    if left_node[0] == self.__class__.LEAF_TYPE and right_node[0] == self.__class__.LEAF_TYPE:
                        if left_node[1] == right_node[1]:
                            # сливаем листы, оставляя узел с меньшим id
                            # основной класс
                            main_class = left_node[1]
                            # кол-во образцов в левом и правом узлах
                            y_size_left = left_node[4]
                            y_size_right = right_node[4]
                            # доля основного класса в левом и правом узлах
                            p_left = left_node[2]
                            p_right = right_node[2]
                            # количество образцов основного класса в левом и правом узлах
                            y_max_left = p_left * y_size_left
                            y_max_right = p_right * y_size_right
                            
                            y_max_merge = y_max_left + y_max_right
                            y_size_merge = y_size_left + y_size_right
                            p_merge = y_max_merge / y_size_merge
                            
                            node_merge = dict()
                            node_merge[0] = self.__class__.LEAF_TYPE
                            node_merge[1] = main_class
                            node_merge[2] = p_merge
                            node_merge[3] = 7
                            node_merge[4] = y_size_merge
                            
                            self.tree[left_node_id] = node_merge
                            self.tree.pop(right_node_id)
                    else:
                        if left_node[0] == self.__class__.NON_LEAF_TYPE:
                            self.__walk_tree_and_merge_leafs(left_node_id)
                        if right_node[0] == self.__class__.NON_LEAF_TYPE:
                            self.__walk_tree_and_merge_leafs(right_node_id)
    
            
    def fit(self, x, y):
        self.num_class = np.unique(y).size  # количество классов признаков. В нашем случае 2 класса - 0 или 1
        self.__fit_node(x, y, 0, 0) 
        # self.__merge_leafs()

        
    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold, _, _ = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

        
    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold, _, _ = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    
    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)
    
    
    def printtree(self, indent=''):
        self.__print_node(0, '')
    
    
    def __print_node(self, node_id, indent):
        if node_id in self.tree:
            indent += ' '
            
            node = self.tree[node_id]
            if node[0] == self.__class__.NON_LEAF_TYPE:    
                print('{} {}: feature_id={}, threshold={}, min_impurity={}, impurity_list={}'.format(indent, node_id, node[1], node[2], node[3], node[4]))
                self.__print_node(2 * node_id + 1, indent)
                self.__print_node(2 * node_id + 2, indent)
            if node[0] == self.__class__.LEAF_TYPE:
                print('{} {}: reason={}, class={}, p={}'.format(indent, node_id, node[3], node[1], node[2]))
        else:
            return

In [3]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
x = df.as_matrix(columns=df.columns[1:])  # столбцы, кроме первого - определяющие признаки
y = df.as_matrix(columns=df.columns[:1])  # первый столбец (SeriousDlqin2yrs) - целевой признак
y = y.reshape(y.shape[0])

In [15]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, max_depth=None)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [6]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.3548545837402344
0.8041176795959473


## Проверка качества работы

In [16]:
np.random.seed(1)

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [19]:
my_clf.fit(X_train, y_train)
y_pred = my_clf.predict(X_test)
y_true = y_test
acc_score = accuracy_score(y_pred, y_true)
print(acc_score)
my_clf.printtree()

0.0688106957563
  0: feature_id=6, threshold=1.0, min_impurity=0.11545043472782135, impurity_list=[0.12036810967891284, 0.12853653703471113, 0.12205564262152974, 0.12947286877977268, 0.1291390355854499, 0.12915201411403296, 0.12117844930940647, 0.12878363545054777, 0.12946137410081687, 0.11545043472782135]
   1: feature_id=6, threshold=3.0, min_impurity=0.48281813994286515, impurity_list=[0.48995906302167441, 0.48830085660214367, 0.491952007853795, 0.48764428203055077, 0.48365115367081335, 0.49159355637961272, 0.49091282897751642, 0.4841040702227577, 0.48774335298009042, 0.48281813994286515]
    3: feature_id=1, threshold=36.0, min_impurity=0.4398715583794493, impurity_list=[0.45275223244035717, 0.45255211873541679, 0.45422914046189389, 0.45110808218486542, 0.4398715583794493, 0.45538926225175569, 0.45027727662934453, 0.45174220384364383, 0.45311653116531175, 0.45419464986112135]
     7: feature_id=8, threshold=0.0, min_impurity=0.39946394065947766, impurity_list=[0.40919691133030456, 

In [40]:
gkf = KFold(n_splits=5, shuffle=True)

In [44]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    
    y_pred=my_clf.predict(X_test)
    y_true=y_test
    acc_score = accuracy_score(y_pred, y_true)
    
    print(acc_score)

0.0699675729608
0.0653113827222
0.929076245115
0.929990853912
0.0712593023739


К сожалению, не получилось найти ошибку. Явно виден какой-то "перескок" при определении класса. Т.е. в каких-то случаях классы предсказываются с точностью до наоборот. У меня ощущение, что это может быть связано с какой-то ошибкой округления при определнии порога, но непонятно...

In [13]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.892076161969
0.895817743411
0.890870541282
0.89220088135
0.89356836985


# Применить для задачи Titanic 

In [14]:
df_train = pd.read_csv('train.csv', index_col='PassengerId')
df_test = pd.read_csv('test.csv', index_col='PassengerId')

In [15]:
df_train = df_train.dropna()
df_test = df_train.dropna()

In [16]:
df_train = df_train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']]
df_test = df_test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]
df_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
PassengerId,,,,,,,
2,1,female,38.0,1,0,71.2833,1
4,1,female,35.0,1,0,53.1000,1
7,1,male,54.0,0,0,51.8625,0
11,3,female,4.0,1,1,16.7000,1
12,1,female,58.0,0,0,26.5500,1


In [17]:
df_train['Sex'] = df_train['Sex'].map({'male': 0, 'female': 1})
df_test['Sex'] = df_test['Sex'].map({'male': 0, 'female': 1})

In [18]:
X_train = df_train.drop('Survived', axis=1).as_matrix()
y_train = df_train['Survived'].as_matrix()

X_test = df_test.as_matrix()

In [19]:
model = MyDecisionTreeClassifier(min_samples_split=2, max_depth=None)

In [20]:
model.fit(X_train, y_train)

In [21]:
y_predict = model.predict(X_test)
print(accuracy_score(y_train, y_predict))

0.672131147541


In [22]:
model.printtree()

  0: feature_id=1, threshold=0.5
   1: feature_id=5, threshold=10.5
    3: reason=5, class=1, p=0.963414634146
    4: feature_id=0, threshold=2.0
     9: reason=3, class=0, p=1.0
     10: reason=4, class=1, p=0.75
   2: feature_id=2, threshold=17.5
    5: reason=5, class=0, p=0.620689655172
    6: reason=3, class=1, p=1.0


Дерево получилось неглубокое. Видим, что оно делится по полу (feature_id = 1), и дальше по возрасту и стоимости билета. Точность предсказания не слишком высокая, наверное потому, что учитывается мало признаков.